In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
df_movies = pd.read_csv('Datasets/ml-latest-small/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [32]:
df_ratings = pd.read_csv('Datasets/ml-latest-small/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [33]:
nm = df_movies['title'].shape[0]
nu = df_ratings['userId'].unique().shape[0]
print("Number of movies:", nm)
print("Number of users:", nu)

Number of movies: 9742
Number of users: 610


In [45]:
num_features = 10
X = np.zeros((nm, num_features))
W = np.zeros((nu, num_features))
b = np.zeros((nu,))
R = np.zeros((nm, nu))
Y = np.zeros((nm, nu))
userId = df_ratings['userId']
movieId = df_ratings['movieId']
rating = df_ratings['rating']
for i in range(len(userId)):
    if (movieId[i] > nm):
        continue
    m_Id = movieId[i]
    u_Id = userId[i]
    Y[m_Id - 1][u_Id - 1] = rating[i]
    R[m_Id - 1][u_Id - 1] = 1

In [51]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    nm, nu = Y.shape
    J = 0

    for j in range(nu):
        for i in range(nm):
            J += R[i][j]*(np.dot(W[j],X[i])+b[0][j] - Y[i][j])**2
    J = 0.5*J
    
    reg_w = 0
    for j in range(nu):
        for k in range(X.shape[1]):
            reg_w += W[j, k]**2
    J += lambda_*0.5*reg_w
            
    reg_x = 0
    for i in range(nm):
        for k in range(X.shape[1]):
            reg_x += X[i, k]**2
    J += lambda_*0.5*reg_x

    return J

In [52]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [49]:

movieList = df_movies['title']
movieList_df = df_movies

my_ratings = np.zeros(nm)

my_ratings[2700] = 5   # Toy Story 3 (2010)
my_ratings[2609] = 2   # Persuasion (2007)
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}')


New user ratings:

Rated 5.0 for  New York Cop (Nyû Yôku no koppu) (1993)
Rated 5.0 for  Blink (1994)
Rated 2.0 for  Cowboy Way, The (1994)
Rated 5.0 for  Nutty Professor, The (1996)
Rated 5.0 for  Die Hard (1988)
Rated 5.0 for  Raging Bull (1980)
Rated 3.0 for  Koyaanisqatsi (a.k.a. Koyaanisqatsi: Life Out of Balance) (1983)
Rated 5.0 for  Romy and Michele's High School Reunion (1997)
Rated 2.0 for  Son of the Sheik, The (1926)
Rated 5.0 for  Mission: Impossible II (2000)
Rated 3.0 for  Fighting Seabees, The (1944)
Rated 1.0 for  Bikini Beach (1964)
Rated 1.0 for  Slumber Party Massacre III (1990)


In [ ]:
# Reload ratings
Y, R = load_ratings_small()

# Add new user ratings to Y
Y = np.c_[my_ratings, Y]

# Add new user indicator matrix to R
R = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)